## Basic Setup

In [1]:
# connect to google drive
import os
import numpy as np

# mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Check for GPU free memory
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
import psutil
import humanize
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print('='*40)
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  print('='*40)
printm() 

Gen RAM Free: 12.9 GB  | Proc size: 120.9 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [4]:
# change root directory such that models are saved in google drive during training
root_dir = "/content/gdrive/My Drive/NLP/Git_MT"
os.chdir(root_dir)
!ls

en_es_data		   Machine_Translation_Mult_Attn.ipynb
Machine_Translation.ipynb  NMT-vanila-LSTM.ipynb


In [27]:
# github setup ()
if '.git' not in os.listdir():
  !git init
  #print("Git init")
  
!git config --global user.email "rajjjabiswas@gmail.com"
!git config --global user.name "Raja Biswas"

!git add -A
!git commit -m "first commit"
!git remote add origin https://github.com/rbiswasfc/Machine-Translation.git



Initialized empty Git repository in /content/gdrive/My Drive/NLP/Git_MT/.git/
[master (root-commit) 1e28f77] first commit
 10 files changed, 551079 insertions(+)
 create mode 100644 Machine_Translation.ipynb
 create mode 100644 Machine_Translation_Mult_Attn.ipynb
 create mode 100644 NMT-vanila-LSTM.ipynb
 create mode 100644 en_es_data/dev.en
 create mode 100644 en_es_data/dev.es
 create mode 100644 en_es_data/test.en
 create mode 100644 en_es_data/test.es
 create mode 100644 en_es_data/train.en
 create mode 100644 en_es_data/train.es
 create mode 100644 en_es_data/vocab.json
fatal: could not read Username for 'https://github.com': No such device or address


In [29]:
% git push -u origin master


UsageError: Line magic function `%git` not found.


In [22]:
!git push -u origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [0]:
!rm -rf .git

In [21]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   NMT-vanila-LSTM.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


## Imports

In [0]:
import math
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

from collections import Counter, namedtuple
from docopt import docopt
from itertools import chain
import json
from typing import List, Tuple, Dict, Set, Union

from docopt import docopt
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from tqdm import tqdm

In [6]:
if True:
  !ls

en_es_data		   Machine_Translation_Mult_Attn.ipynb
Machine_Translation.ipynb  NMT-vanila-LSTM.ipynb


False